In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import re

Удаляем первые 6 стобцов и заменяем значения "н/д" на NaN (Not a Number), т.к. с ними удобнее работать в коде. Удалим столбцы, которые считаем неинтересными, а именно количество квартир того или иного типа (но оставим их долю).

In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_excel('analys.xlsx')
df = df.iloc[:, 7:]
df.replace(to_replace='^[нН]/д$', value=None, regex=True, inplace=True)
df.drop(["Высота типового этажа пол-пол", "4кк и более", "Студии"], axis=1, inplace=True)
df.drop(df.filter(regex=r'^[0-9]кк$', axis=1).columns, axis=1, inplace=True)

C:\Users\misha\AppData\Local\Temp\ipykernel_27804\3196889945.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(to_replace='^[нН]/д$', value=None, regex=True, inplace=True)


Удалим столбцы, в которых больше 10 пропущенных значений, так как они будут портить представление о данных.

In [3]:
df.dropna(axis=1, thresh=len(df) - 10, inplace=True)

In [4]:
df.head(20)

,Класс,Продаваемая площадь квартир,Количество квартир,Средняя площадь квартиры,Студии д,1кк д,2кк д,3кк д,4кк д,Массинг,Конструктив здания,Конструктив фасада,Материалы облицовки фасада,Подсветка фасада,Размещение внешних блоков кондиционеров,Подготовка под кондиционирование,Угловые окна,Окна в пол,Панорамные окна,Моллированные,Широкие окна,Наиболее частая ширина типовых окон,Наиболее частая высота типовых окон,Коэффициент летних помещений,Французский балкон,Открытый балкон с металлическими ограждениями,Открытый балкон с прозрачными ограждениями,Закрытый балкон,Лоджия,Терраса на 1 эт (палисадник),Терраса на среднем/типовом этаже,Терраса на кровле,Отделка лобби,Меблировка лобби,Наличие центрального / гранд-лобби,Колясочные,Велосипедные,Лапомойки,Зона ожидания,Стойка Reception,Гостевой с/у,Детская игровая зона,Коворкинг,Кофе-поинт,Постамат,Переговорная,Отделка типового этажа (описание / концепция): межквартирный холл и лифтовой коридор,Гостевые С/У,Двусветное лобби,Кол-во лифтов на 1 подъезд,Опускание лифта в паркинг,Высота типового этажа пол-потолок,Высота верхних этажей пол-потолок,Глубина квартир,Система противопожарной безопасности,Тип паркинга,Количество м/м,Обеспеченность машиноместами,количество кладовок,Обеспеченность кладовыми,Продаваемая площадь кладовых,Средняя площадь кладовых,Тип разводки отопления,Тип разводки воды,Отделка квартир,Стоимость полной отделки,Шумоизоляция пола,Мусороудаление,Бесперебойное горячее водоснабжение/отопление,Фильтрация воды,"Система усиления GSM сигнала (лифты, паркинг и тд)",Вызывная панель / домофон,Зона Reception,Помещения ГБР,Закрытый двор,Площадка для баскетбола (наличие),Футбольная площадка (наличие),Столы для настольного тенниса (наличие),Велодорожки,Пешеходный бульвар,Благоустроенная набережная,Тип МАФ,PlayHub,Local Play,SportHub,DogHub (площадка для выгула собак),Рабочие места во дворе,WI-FI на территории,Эксплуатируемая кровля,"Площадь БКФН на 1 этажах, кв.м",Количество БКФН,Средняя площадь БКФН,Доля БКФН в составе ЖК,Торговый центр,Бизнес-центр,Light-industrial
0,Бизнес,10449.00,92,113.576087,0.000000,0.173913,0.250000,0.369565,0.206522,"Умеренный масинг (раскреповки, небольшие сдвиги)",Монолит,НВФ - навесной вентилируемый фасад,Фиброцементные плиты,Да,Места для внешних блоков на балконах,Подготовка отсутствует,True,True,False,False,True,2200.0,2100.0,20-30%,False,False,False,True,True,True,False,False,Бизнес - дизайн с натуральными материалами + э...,"Да, мягкая мебель, зона ожидания",да,True,False,True,True,True,True,False,False,False,False,False,"Типовая (штукатурка стен, керамическое покрыти...",Нет,Нет,2,Частично,2950,3050,7200,Датчики задымления,Подземный,906,9.847826,83,0.902174,446.82,5.383373,Коллекторная/горизонтальная,Коллекторная/горизонтальная,WB,0.0,Да,ТБО встроенный на 1 этаже,Да (ИТП),Общедомовая очистка,Да,На всех этажах,Да,Да,частично,Нет,Нет,Да,Да,Да,Да,Индивидуальные,Да,Да,Да,Нет,Нет,Да,Нет,627.00,9.0,69.666667,0.097826,True,True,True
1,Бизнес,10439.00,101,103.356436,0.000000,0.069307,0.504950,0.237624,0.188119,"Умеренный масинг (раскреповки, небольшие сдвиги)",Монолит,НВФ - навесной вентилируемый фасад,Фиброцементные плиты,Да,Ниши на фасадах под кондиционеры,Подготовка отсутствует,True,False,True,False,True,2100.0,2000.0,20-30%,False,True,True,True,True,True,False,False,Бизнес - дизайн с натуральными материалами + э...,"Да, мягкая мебель, зона ожидания",да,True,False,True,True,True,False,False,False,False,False,False,"Типовая (штукатурка стен, керамическое покрыти...",Нет,Нет,2,Частично,3000,3050,6900,Датчики задымления,Подземный,634,6.277228,83,0.821782,446.82,5.383373,Коллекторная/горизонтальная,Коллекторная/горизонтальная,WB,0.0,Да,ТБО встроенный на 1 этаже,Да (ИТП),Общедомовая очистка,Да,На всех этажах,Да,Да,частично,Нет,Нет,Да,Да,Да,Да,Индивидуальные,Да,Да,Да,Нет,Нет,Да,Нет,774.00,11.0,70.363636,0.108911,True,True,True
2,Комфорт,86718.00,1506,57.581673,0.000000,0.183267,0.439575,0.318725,0.058433,"""Коробка"" - нет массинга",Монолит,НВФ - навесной вентили

Удалим строки, в которых остались пропущенные значения, т.к. их нельзя использовать при классификации.

In [5]:
df.dropna(inplace=True)

Смотрим, в каких столбцах есть одновременно строки и числа.

In [6]:
def has_mixed_data(column):
    return any(isinstance(value, str) for value in column) and any(isinstance(value, (int, float)) for value in column)

mixed_columns = [column for column in df.columns if has_mixed_data(df[column])]

In [7]:
columns_to_process = mixed_columns

def replace_range_value(value):
    match = re.search(r'\d+', value)
    if match:
        return int(match.group())
    else:
        return value

for column in columns_to_process:
    df[column] = df[column].apply(lambda x: replace_range_value(x) if isinstance(x, str) else x)

In [8]:
df.head(20)

,Класс,Продаваемая площадь квартир,Количество квартир,Средняя площадь квартиры,Студии д,1кк д,2кк д,3кк д,4кк д,Массинг,Конструктив здания,Конструктив фасада,Материалы облицовки фасада,Подсветка фасада,Размещение внешних блоков кондиционеров,Подготовка под кондиционирование,Угловые окна,Окна в пол,Панорамные окна,Моллированные,Широкие окна,Наиболее частая ширина типовых окон,Наиболее частая высота типовых окон,Коэффициент летних помещений,Французский балкон,Открытый балкон с металлическими ограждениями,Открытый балкон с прозрачными ограждениями,Закрытый балкон,Лоджия,Терраса на 1 эт (палисадник),Терраса на среднем/типовом этаже,Терраса на кровле,Отделка лобби,Меблировка лобби,Наличие центрального / гранд-лобби,Колясочные,Велосипедные,Лапомойки,Зона ожидания,Стойка Reception,Гостевой с/у,Детская игровая зона,Коворкинг,Кофе-поинт,Постамат,Переговорная,Отделка типового этажа (описание / концепция): межквартирный холл и лифтовой коридор,Гостевые С/У,Двусветное лобби,Кол-во лифтов на 1 подъезд,Опускание лифта в паркинг,Высота типового этажа пол-потолок,Высота верхних этажей пол-потолок,Глубина квартир,Система противопожарной безопасности,Тип паркинга,Количество м/м,Обеспеченность машиноместами,количество кладовок,Обеспеченность кладовыми,Продаваемая площадь кладовых,Средняя площадь кладовых,Тип разводки отопления,Тип разводки воды,Отделка квартир,Стоимость полной отделки,Шумоизоляция пола,Мусороудаление,Бесперебойное горячее водоснабжение/отопление,Фильтрация воды,"Система усиления GSM сигнала (лифты, паркинг и тд)",Вызывная панель / домофон,Зона Reception,Помещения ГБР,Закрытый двор,Площадка для баскетбола (наличие),Футбольная площадка (наличие),Столы для настольного тенниса (наличие),Велодорожки,Пешеходный бульвар,Благоустроенная набережная,Тип МАФ,PlayHub,Local Play,SportHub,DogHub (площадка для выгула собак),Рабочие места во дворе,WI-FI на территории,Эксплуатируемая кровля,"Площадь БКФН на 1 этажах, кв.м",Количество БКФН,Средняя площадь БКФН,Доля БКФН в составе ЖК,Торговый центр,Бизнес-центр,Light-industrial
0,Бизнес,10449.00,92,113.576087,0.000000,0.173913,0.250000,0.369565,0.206522,"Умеренный масинг (раскреповки, небольшие сдвиги)",Монолит,НВФ - навесной вентилируемый фасад,Фиброцементные плиты,Да,Места для внешних блоков на балконах,Подготовка отсутствует,True,True,False,False,True,2200.0,2100.0,20-30%,False,False,False,True,True,True,False,False,Бизнес - дизайн с натуральными материалами + э...,"Да, мягкая мебель, зона ожидания",да,True,False,True,True,True,True,False,False,False,False,False,"Типовая (штукатурка стен, керамическое покрыти...",Нет,Нет,2,Частично,2950,3050,7200,Датчики задымления,Подземный,906,9.847826,83,0.902174,446.820000,5.383373,Коллекторная/горизонтальная,Коллекторная/горизонтальная,WB,0.0,Да,ТБО встроенный на 1 этаже,Да (ИТП),Общедомовая очистка,Да,На всех этажах,Да,Да,частично,Нет,Нет,Да,Да,Да,Да,Индивидуальные,Да,Да,Да,Нет,Нет,Да,Нет,627.00,9.0,69.666667,0.097826,True,True,True
1,Бизнес,10439.00,101,103.356436,0.000000,0.069307,0.504950,0.237624,0.188119,"Умеренный масинг (раскреповки, небольшие сдвиги)",Монолит,НВФ - навесной вентилируемый фасад,Фиброцементные плиты,Да,Ниши на фасадах под кондиционеры,Подготовка отсутствует,True,False,True,False,True,2100.0,2000.0,20-30%,False,True,True,True,True,True,False,False,Бизнес - дизайн с натуральными материалами + э...,"Да, мягкая мебель, зона ожидания",да,True,False,True,True,True,False,False,False,False,False,False,"Типовая (штукатурка стен, керамическое покрыти...",Нет,Нет,2,Частично,3000,3050,6900,Датчики задымления,Подземный,634,6.277228,83,0.821782,446.820000,5.383373,Коллекторная/горизонтальная,Коллекторная/горизонтальная,WB,0.0,Да,ТБО встроенный на 1 этаже,Да (ИТП),Общедомовая очистка,Да,На всех этажах,Да,Да,частично,Нет,Нет,Да,Да,Да,Да,Индивидуальные,Да,Да,Да,Нет,Нет,Да,Нет,774.00,11.0,70.363636,0.108911,True,True,True
9,Стандарт,21608.00,406,53.221675,0.000000,0.197044,0.487685,0.275862,0.039409,"Умеренный масинг (раскреповки, небольшие сдвиги)",

In [9]:
df['Класс'].unique()

array(['Бизнес', 'Стандарт', 'Комфорт', 'Элит', 'Премиум'], dtype=object)

In [10]:
label = LabelEncoder()
for i in df.select_dtypes("object").columns:
    df[i] = label.fit_transform(df[i])
print(df)

     Класс  Продаваемая площадь квартир  Количество квартир  \
0        0                      10449.0                  92   
1        0                      10439.0                 101   
9        3                      21608.0                 406   
10       3                      19943.0                 366   
11       3                       8781.0                 170   
..     ...                          ...                 ...   
131      0                       7300.8                  96   
132      0                       7476.9                  64   
133      1                       3626.0                  88   
134      1                       3308.0                  79   
135      1                       3662.0                  88   

     Средняя площадь квартиры  Студии д     1кк д     2кк д     3кк д  \
0                  113.576087       0.0  0.173913  0.250000  0.369565   
1                  103.356436       0.0  0.069307  0.504950  0.237624   
9                   53.2

## Классификация

In [11]:
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold

Подготавливаем данные для классификации.

Первый столбец ("Класс") - целевое значение.

Остальные столбцы - признаки, по которым мы будем предсказывать класс.

In [12]:
x = df.iloc[:, 1:]
y = df.iloc[:, 0]

x = np.array(x)
y = np.array(y)

### Дерево решений

In [13]:
model_tree = DecisionTreeClassifier(random_state=42, max_depth=5, min_samples_split=5, min_samples_leaf=5, criterion='gini')

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_results = cross_validate(model_tree, x, y, cv=cv, scoring=['accuracy', 'precision_macro', 'recall_macro'], return_train_score=True, return_estimator=True)

print(f"Оценка точности: {(cv_results['test_accuracy'].mean()*100):.3f}%")
print(f"Максимальная точность: {(cv_results['test_accuracy'].max()*100):.3f}%")
print("Оценка точности (обучение):", cv_results['train_accuracy'].mean())
print("Оценка precision (macro):", cv_results['test_precision_macro'].mean())
print("Оценка recall (macro):", cv_results['test_recall_macro'].mean())
print("Максимальный precision (macro):", cv_results['test_precision_macro'].max())
print("Максимальный recall (macro):", cv_results['test_recall_macro'].max())
print("Время обучения (среднее):", cv_results['fit_time'].mean())
print("Время предсказания (среднее):", cv_results['score_time'].mean())

Оценка точности: 83.913%
Максимальная точность: 95.455%
Оценка точности (обучение): 0.9031664964249233
Оценка precision (macro): 0.6761666666666667
Оценка recall (macro): 0.7404444444444445
Максимальный precision (macro): 0.9444444444444444
Максимальный recall (macro): 0.95
Время обучения (среднее): 0.0017433643341064453
Время предсказания (среднее): 0.0018110275268554688


d:\prikoli\.venv\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
d:\prikoli\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\prikoli\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\prikoli\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `ze

Выбираем лучшую модель

In [14]:
best_model_index = cv_results['test_accuracy'].argmax()
best_model = cv_results['estimator'][best_model_index]

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y, shuffle=True)

y_pred = best_model.predict(x_test)

print('Accuracy: ', accuracy_score(y_test, y_pred))

Accuracy:  0.9130434782608695
